# <b>Smooke Test</b>

Running queries in created tables from:

* Generated temporal data
* S3 data
* GS data

In [ ]:
## <b>Init Context</b>

In [ ]:
import dask

In [ ]:
from dask.distributed import Client

In [ ]:
ip_port_dask_scheduller = 'SCHEDULER_IP:8786'
network_interface = 'ens5' 

In [ ]:
client = Client(ip_port_dask_scheduller)

In [ ]:
client

In [ ]:
from blazingsql import BlazingContext

In [ ]:
bc = BlazingContext(dask_client = client, network_interface='ens5')

In [ ]:
create_logger()

## <b>Generated temporal data</b>

In [ ]:
import cudf

In [ ]:
num_partitions=2
data_size=1000

In [ ]:
df2 = cudf.DataFrame()

In [ ]:
df2['col1'] = cudf.Series(list(range(data_size)))

In [ ]:
df2['col2'] = cudf.Series(list(range(data_size)))

In [ ]:
import dask_cudf

In [ ]:
ds2 = dask_cudf.from_cudf(df2,npartitions=num_partitions)

In [ ]:
df = cudf.DataFrame()

In [ ]:
df['col1'] = cudf.Series(list(range(data_size)))

In [ ]:
df['col2'] = cudf.Series(list(range(data_size)))

In [ ]:
ds = dask_cudf.from_cudf(df,npartitions=num_partitions)

In [ ]:
t = bc.create_table("dask_table",ds)

In [ ]:
t = bc.create_table("dask_table2",ds2)

In [ ]:
test = bc.sql('select * from dask_table inner join dask_table2 on dask_table2.col1 = dask_table.col1')

In [ ]:
print(test.head(10))

## <b>Data from S3</b>

In [ ]:
from blazingsql import S3EncryptionType

In [ ]:
# Here you have to set values of environment variables

os.environ['BLAZINGSQL_E2E_AWS_S3_ACCESS_KEY_ID']=''
os.environ['BLAZINGSQL_E2E_AWS_S3_SECRET_KEY']=''

In [ ]:
# Getting values of environment variables

access_key_id = os.getenv("BLAZINGSQL_E2E_AWS_S3_ACCESS_KEY_ID")
secret_key = os.getenv("BLAZINGSQL_E2E_AWS_S3_SECRET_KEY")

In [ ]:
authority = "tpch_s3"

In [ ]:
bc.s3(authority, bucket_name='blazingsql-bucket', encryption_type=S3EncryptionType.NONE,
	access_key_id=access_key_id, secret_key=secret_key)

In [ ]:
dir_data_lc = "s3://" + authority + "/" + "DataSet100Mb2part/" + "tpch/"

In [ ]:
ext = "parquet"

In [ ]:
table_files_nation = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "nation", ext)

In [ ]:
table_files_region = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "region", ext)

In [ ]:
print("nation files: " + table_files_nation)

In [ ]:
bc.create_table("nation", table_files_nation)
print("nation table created!")

In [ ]:
bc.create_table("region", table_files_region)
print("region table created!")

In [ ]:
query = """select n1.n_nationkey as n1key, n2.n_nationkey as n2key, n1.n_nationkey + n2.n_nationkey 
from nation as n1 full outer join nation as n2 on n1.n_nationkey = n2.n_nationkey + 6 
where n1.n_nationkey < 10 and n1.n_nationkey > 5"""

In [ ]:
test = bc.sql(query)

In [ ]:
print(test.compute())